A CNN model is created and trained on the char54k dataset to identify digits 1 to 9

In [7]:
def getBestShift(img): #function to calculate the COM shift
    cy,cx = ndimage.measurements.center_of_mass(img)
    rows,cols = img.shape
    shiftx = numpy.round(cols/2.0-cx).astype(int)
    shifty = numpy.round(rows/2.0-cy).astype(int)
    return shiftx,shifty

In [8]:
def createTrainingData(): #function is used to read image traning data from the folders named 1, 2,..., 9 in the DigitImages directory
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (img_rows, img_cols))
            new_array = shiftCenterOfMass(new_array)
            training_data.append([new_array, class_num])

In [9]:
#This function will centralize the image according to its center of mass.Same has been implemented in sudokuSolver. Without this step, the accuracy drops from nearly 100% to 70-80%
def shiftCenterOfMass(img):
    img = cv2.bitwise_not(img)

    # Centralize the image according to center of mass
    shiftx,shifty = getBestShift(img)
    shifted = shift(img,shiftx,shifty)
    img = shifted

    img = cv2.bitwise_not(img)
    return img

In [10]:
#function to shift image
def shift(img,sx,sy):
    rows,cols = img.shape
    M = numpy.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical


import numpy
import os
import random
import cv2
from scipy import ndimage

In [12]:
batch_size = 128
num_classes = 9
epochs = 45

# input image dimensions
img_rows, img_cols = 28, 28

DATADIR = "DigitImages"
CATEGORIES = ["1","2","3","4","5","6","7","8","9"]

training_data = []

createTrainingData()

# Mix data up
random.shuffle(training_data)

# Split 80-20
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(training_data)*8//10):
    x_train.append(training_data[i][0])
    y_train.append(training_data[i][1])
for i in range(len(training_data)*8//10,len(training_data)):
    x_test.append(training_data[i][0])
    y_test.append(training_data[i][1])

# Reshape
x_train = numpy.array(x_train)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = numpy.array(x_test)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the data
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer="Adam",metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save_weights('digitRecognition.h5')

C:\Users\Rohan kumar\AppData\Local\Temp\ipykernel_27456\1808188631.py:2: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cy,cx = ndimage.measurements.center_of_mass(img)





Epoch 1/45


58/58 [==============================] - 28s 388ms/step - loss: 0.7769 - accuracy: 0.7699 - val_loss: 0.1797 - val_accuracy: 0.9448
Epoch 2/45
58/58 [==============================] - 25s 429ms/step - loss: 0.2112 - accuracy: 0.9374 - val_loss: 0.1109 - val_accuracy: 0.9645
Epoch 3/45
58/58 [==============================] - 22s 377ms/step - loss: 0.1197 - accuracy: 0.9647 - val_loss: 0.0700 - val_accuracy: 0.9765
Epoch 4/45
58/58 [==============================] - 23s 398ms/step - loss: 0.0835 - accuracy: 0.9748 - val_loss: 0.0518 - val_accuracy: 0.9809
Epoch 5/45
58/58 [==============================] - 23s 396ms/step - loss: 0.0621 - accuracy: 0.9814 - val_loss: 0.0409 - val_accuracy: 0.9852
Epoch 6/45
58/58 [==============================] - 19s 326ms/step - loss: 0.0466 - accuracy: 0.9856 - val_loss: 0.0298 - val_accuracy: 0.9891
Epoch 7/45
58/58 [==============================] - 19s 323ms/step - loss: 0.0401 - accuracy: 0.9880 - val_loss: 0.0306 - val_accuracy: 0